# Penalaan Halus Model Open AI

Buku nota ini berdasarkan panduan terkini yang disediakan dalam dokumentasi [Penalaan Halus](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dari Open AI.

Penalaan halus meningkatkan prestasi model asas untuk aplikasi anda dengan melatih semula menggunakan data tambahan dan konteks yang berkaitan dengan kes penggunaan atau senario tertentu itu. Perlu diingat bahawa teknik kejuruteraan prompt seperti _few shot learning_ dan _retrieval augmented generation_ membolehkan anda meningkatkan prompt lalai dengan data yang relevan untuk memperbaiki kualiti. Walau bagaimanapun, pendekatan ini terhad oleh saiz tetingkap token maksimum model asas yang disasarkan.

Dengan penalaan halus, kita sebenarnya melatih semula model itu sendiri dengan data yang diperlukan (membolehkan kita menggunakan lebih banyak contoh daripada yang boleh muat dalam tetingkap token maksimum) - dan menggunakan versi _custom_ model yang tidak lagi memerlukan contoh disediakan semasa masa inferens. Ini bukan sahaja meningkatkan keberkesanan reka bentuk prompt kita (kita mempunyai lebih fleksibiliti dalam menggunakan tetingkap token untuk perkara lain) tetapi juga berpotensi memperbaiki kos kita (dengan mengurangkan bilangan token yang perlu dihantar ke model semasa masa inferens).

Penalaan halus mempunyai 4 langkah:
1. Sediakan data latihan dan muat naiknya.
1. Jalankan kerja latihan untuk mendapatkan model yang telah ditala halus.
1. Nilai model yang telah ditala halus dan ulangi untuk kualiti.
1. Gunakan model yang telah ditala halus untuk inferens apabila berpuas hati.

Perlu diingat bahawa tidak semua model asas menyokong penalaan halus - [semak dokumentasi OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) untuk maklumat terkini. Anda juga boleh menala halus model yang telah ditala halus sebelum ini. Dalam tutorial ini, kita akan menggunakan `gpt-35-turbo` sebagai model asas sasaran untuk penalaan halus. 

---


### Langkah 1.1: Sediakan Dataset Anda

Mari bina chatbot yang membantu anda memahami jadual berkala unsur dengan menjawab soalan tentang satu unsur menggunakan limerik. Dalam tutorial mudah _ini_, kita hanya akan mencipta dataset untuk melatih model dengan beberapa contoh respons yang menunjukkan format data yang dijangka. Dalam kes penggunaan dunia sebenar, anda perlu mencipta dataset dengan lebih banyak contoh. Anda juga mungkin boleh menggunakan dataset terbuka (untuk domain aplikasi anda) jika ada, dan memformat semula untuk digunakan dalam penalaan halus.

Oleh kerana kita memfokuskan pada `gpt-35-turbo` dan mencari respons satu pusingan (penyempurnaan sembang), kita boleh mencipta contoh menggunakan [format yang dicadangkan ini](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) yang mencerminkan keperluan penyempurnaan sembang OpenAI. Jika anda menjangkakan kandungan perbualan berbilang pusingan, anda akan menggunakan [format contoh berbilang pusingan](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) yang termasuk parameter `weight` untuk menandakan mesej mana yang harus digunakan (atau tidak) dalam proses penalaan halus.

Kita akan menggunakan format satu pusingan yang lebih mudah untuk tutorial kita di sini. Data adalah dalam format [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) dengan 1 rekod setiap baris, setiap satu diwakili sebagai objek berformat JSON. Petikan di bawah menunjukkan 2 rekod sebagai contoh - lihat [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) untuk set contoh penuh (10 contoh) yang akan kita gunakan untuk tutorial penalaan halus kita. **Nota:** Setiap rekod _mesti_ ditakrifkan dalam satu baris sahaja (tidak dibahagi merentasi baris seperti biasa dalam fail JSON yang diformat)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Dalam kes penggunaan dunia sebenar, anda akan memerlukan set contoh yang jauh lebih besar untuk hasil yang baik - pertukaran adalah antara kualiti respons dan masa/kos untuk penalaan halus. Kita menggunakan set kecil supaya kita dapat menyelesaikan penalaan halus dengan cepat untuk menggambarkan proses tersebut. Lihat [contoh OpenAI Cookbook ini](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) untuk tutorial penalaan halus yang lebih kompleks.


---

### Langkah 1.2 Muat Naik Dataset Anda

Muat naik data menggunakan Files API [seperti yang diterangkan di sini](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Perlu diingat bahawa untuk menjalankan kod ini, anda mesti telah melakukan langkah-langkah berikut terlebih dahulu:
 - Pasang pakej Python `openai` (pastikan anda menggunakan versi >=0.28.0 untuk ciri terkini)
 - Tetapkan pembolehubah persekitaran `OPENAI_API_KEY` kepada kunci API OpenAI anda
Untuk mengetahui lebih lanjut, lihat [panduan Persediaan](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) yang disediakan untuk kursus ini.

Sekarang, jalankan kod untuk mencipta fail untuk dimuat naik dari fail JSONL tempatan anda.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Langkah 2.1: Cipta tugasan Penalaan Halus dengan SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Langkah 2.2: Semak Status kerja

Berikut adalah beberapa perkara yang anda boleh lakukan dengan API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Senaraikan n kerja penalaan terakhir
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Dapatkan butiran kerja penalaan tertentu
- `client.fine_tuning.jobs.cancel(<job_id>)` - Batalkan kerja penalaan
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Senaraikan sehingga n acara dari kerja tersebut
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Langkah pertama proses ini adalah _mengesahkan fail latihan_ untuk memastikan data dalam format yang betul.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Langkah 2.3: Jejak acara untuk memantau kemajuan


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Langkah 2.4: Lihat status dalam Papan Pemuka OpenAI


Anda juga boleh melihat status dengan melawat laman web OpenAI dan meneroka bahagian _Fine-tuning_ platform tersebut. Ini akan menunjukkan status kerja semasa, dan juga membolehkan anda menjejaki sejarah pelaksanaan kerja terdahulu. Dalam tangkapan skrin ini, anda boleh melihat bahawa pelaksanaan terdahulu gagal, dan larian kedua berjaya. Untuk konteks, ini berlaku apabila larian pertama menggunakan fail JSON dengan rekod yang diformatkan dengan tidak betul - setelah diperbaiki, larian kedua selesai dengan jayanya dan menjadikan model tersedia untuk digunakan.

![Fine-tuning job status](../../../../../translated_images/ms/fine-tuned-model-status.563271727bf7bfba.webp)


Anda juga boleh melihat mesej status dan metrik dengan menatal ke bawah lagi dalam papan pemuka visual seperti yang ditunjukkan:

| Mesej | Metrik |
|:---|:---|
| ![Messages](../../../../../translated_images/ms/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/ms/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Langkah 3.1: Dapatkan ID & Uji Model Fine-Tuned dalam Kod


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Langkah 3.2: Muat & Uji Model Halus yang Telah Disesuaikan di Playground

Anda kini boleh menguji model halus yang telah disesuaikan dengan dua cara. Pertama, anda boleh melawat Playground dan menggunakan menu lungsur Models untuk memilih model halus yang baru anda sesuaikan daripada pilihan yang disenaraikan. Pilihan lain adalah menggunakan pilihan "Playground" yang ditunjukkan dalam panel Fine-tuning (lihat tangkapan skrin di atas) yang melancarkan paparan _perbandingan_ berikut yang menunjukkan versi model asas dan model halus yang disesuaikan secara berdampingan untuk penilaian cepat.

![Fine-tuning job status](../../../../../translated_images/ms/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Isikan sahaja konteks sistem yang digunakan dalam data latihan anda dan berikan soalan ujian anda. Anda akan perasan bahawa kedua-dua sisi dikemas kini dengan konteks dan soalan yang sama. Jalankan perbandingan dan anda akan melihat perbezaan output antara keduanya. _Perhatikan bagaimana model halus yang disesuaikan menghasilkan respons dalam format yang anda berikan dalam contoh anda manakala model asas hanya mengikuti arahan sistem_.

![Fine-tuning job status](../../../../../translated_images/ms/fine-tuned-playground-launch.5a26495c983c6350.webp)

Anda akan perasan bahawa perbandingan juga menyediakan kiraan token untuk setiap model, dan masa yang diambil untuk inferens. **Contoh khusus ini adalah contoh ringkas yang bertujuan untuk menunjukkan proses tetapi tidak mencerminkan dataset atau senario dunia sebenar**. Anda mungkin perasan bahawa kedua-dua sampel menunjukkan bilangan token yang sama (konteks sistem dan arahan pengguna adalah sama) dengan model halus yang disesuaikan mengambil masa lebih lama untuk inferens (model tersuai).

Dalam senario dunia sebenar, anda tidak akan menggunakan contoh mainan seperti ini, tetapi menyesuaikan model dengan data sebenar (contohnya, katalog produk untuk perkhidmatan pelanggan) di mana kualiti respons akan lebih ketara. Dalam _konteks_ itu, mendapatkan kualiti respons yang setara dengan model asas akan memerlukan kejuruteraan arahan tersuai yang lebih banyak yang akan meningkatkan penggunaan token dan berpotensi masa pemprosesan yang berkaitan untuk inferens. _Untuk mencuba ini, lihat contoh fine-tuning dalam OpenAI Cookbook untuk memulakan._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang sahih. Untuk maklumat penting, terjemahan profesional oleh manusia adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
